## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

#funkcja z zadania podstawowego
def canny1(I, TH, TL):
    X, Y = I.shape
    blur=cv2.GaussianBlur(I, (5, 5), 0)
    S1 = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    S2 = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])
    Gx=cv2.filter2D(blur, -1, S1)
    Gy=cv2.filter2D(blur, -1, S2)
    OW=np.sqrt(Gx**2+Gy**2)
    alfa_=np.arctan2(Gy, Gx)
    alfa = alfa_*180./np.pi
    alfa[alfa < 0] += 180
    direction=np.zeros((X,Y))
    for i in range (X):
        for j in range (Y):
            if (0 <= alfa[i,j] < 22.5) or (157.5 <= alfa[i,j] <= 180):
                direction[i,j] = 1
            elif (22.5 <= alfa[i,j] < 67.5):
                direction[i,j] = 2
            elif (67.5 <= alfa[i,j] < 112.5):
                direction[i,j] = 3
            elif (112.5 <= alfa[i,j] < 157.5):
                direction[i,j] = 4
    def nonmax(direction, OW):
        X,Y = direction.shape
        gN = np.zeros((X,Y))
        for i in range (1, X - 1):
            for j in range (1, Y - 1):
                if(direction[i, j] == 1):
                    if(OW[i, j - 1] > OW[i, j] or OW[i, j + 1] > OW[i, j]):
                        gN[i, j] = 0
                    else:
                        gN[i, j] = OW[i, j]
                elif(OW[i, j] == 2):
                    if(OW[i + 1, j - 1] > OW[i, j] or OW[i - 1, j + 1] > OW[i, j]):
                        gN[i, j] = 0
                    else:
                        gN[i, j] = OW[i, j]
                elif(direction[i, j] == 3):
                    if(OW[i + 1, j] > OW[i, j] or OW[i - 1, j] > OW[i, j]):
                        gN[i, j] = 0
                    else:
                        gN[i, j] = OW[i, j]
                elif(direction[i, j] == 4):
                    if(OW[i - 1, j - 1] > OW[i, j] or OW[i + 1, j + 1] > OW[i, j]):
                        gN[i, j] = 0
                    else:
                        gN[i, j] = OW[i ,j]
        return gN
    gN = nonmax(direction ,OW)
    gNH = gN >= TH
    gNL = np.where(np.logical_and(TH > gN, gN >= TL), 1, 0)
    return gNH, gNL


def canny2(I, gNH, gNL):
    X,Y = I.shape
    list = []
    for i in range(1, X - 1):
        for j in range(1, Y - 1):
            if gNH[i, j] == 1:
                list.append([i, j])
    visited = np.zeros((X, Y))
    edges  = np.zeros((X, Y))
    while(list):
        element = list.pop()
        x = element[0]
        y = element[1]
        if visited[x, y] == 0:
            visited[x, y] = 1
            edges[x, y] = 1
            for m in range(x - 1, x + 2):
                for n in range(y - 1, y + 2):
                    if(gNL[m, n] == 1):
                        list.append([m, n])
    return edges

gNH, gNL = canny1(house, TL=5, TH=10)
canny = canny2(I=house, gNH=gNH, gNL=gNL)
canny_cv2 = cv2.Canny(house, 5, 10, None, 3, 1)

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 25))
ax1.imshow(house, 'gray')
ax1.set_title("Obraz oryginalny")
ax1.axis('off')
ax2.imshow(gNH, 'gray')
ax2.set_title("gNH")
ax2.axis('off')
ax3.imshow(canny, 'gray')
ax3.set_title("Obraz wynikowy")
ax3.axis('off')
ax4.imshow(canny_cv2, 'gray')
ax4.set_title("Obraz wynikowy cv2")
ax4.axis('off')